In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import pandas as pd
import os
import sys

sys.path.append('../../utils')
import config_handling as conf
from database import Database
from file_io import path_handler
from tqdm import tqdm

In [2]:
use_bbox = True    #run the model/training with bounding boxes when available? 

In [3]:
# Load configuration
config = conf.read_config('../../config/automotive.conf.ini')
config.read('config.ini')
connection_type = config['settings']['connection']
user = config[connection_type]['user']
pw = config[connection_type]['pw']
host = config[connection_type]['host']
db = config[connection_type]['db']
port = config[connection_type].getint('port')
db = Database(host,
              port,
              user,
              pw,
              db
              )
db.connect()

Connection established


In [4]:
# Image directory
basedir = config['settings']['image_directory']

In [5]:
distinct_brands = db.execute_query('SELECT distinct(brand) FROM listings')

In [6]:
brands = pd.DataFrame(distinct_brands)

In [7]:
BRANDS = brands['brand']


In [8]:

def bbox_to_ints(df):
    df = df.copy()
    for col in ['yolobox_top_left_x', 'yolobox_top_left_y', 'yolobox_bottom_right_x', 'yolobox_bottom_right_y']:
        df[col] = df[col].astype(int)
    return df

In [9]:
# Query data
tags = db.execute_query("""SELECT
                            images.yolobox_top_left_x, 
                            images.yolobox_top_left_y, 
                            images.yolobox_bottom_right_x, 
                            images.yolobox_bottom_right_y, 
                            images.image_path, 
                            listings.brand
                        FROM images 
                        JOIN listings ON listings.id = images.listing_id
                        WHERE 
                            listings.countrycode = 'B' """)
labeled_data = pd.DataFrame(tags).fillna(-1)
labeled_data = bbox_to_ints(labeled_data)


In [10]:
labeled_data["image_path"] = labeled_data["image_path"].apply(lambda x: x.replace('\\', '/'))
labeled_data["abs_image_path"]= basedir + '/' + labeled_data["image_path"]

In [11]:
len(labeled_data)

1354804

In [12]:
labeled_data.sample(5)

,yolobox_top_left_x,yolobox_top_left_y,yolobox_bottom_right_x,yolobox_bottom_right_y,image_path,brand,abs_image_path
500108,115,58,701,493,mercedes-benz/E-Klasse (alle)/baa194fd-93ff-4d...,mercedes-benz,/home/frederic/Documents/automotive_image_data...
475568,115,194,635,421,mercedes-benz/C-Klasse (alle)/2807eb48-1110-42...,mercedes-benz,/home/frederic/Documents/automotive_image_data...
525020,-1,-1,-1,-1,mercedes-benz/V-Klasse (alle)/0d590e79-6621-46...,mercedes-benz,/home/frederic/Documents/automotive_image_data...
336148,-1,-1,-1,-1,volkswagen/T-Roc/b66a18e9-c256-4387-bb03-e0608...,volkswagen,/home/frederic/Documents/automotive_image_data...
161619,91,5,676,546,skoda/Kamiq/1cac471f-8f4d-494c-8394-5401d17b6e...,skoda,/home/frederic/Documents/automotive_image_data...


In [13]:
brand_encoder = LabelEncoder()
labeled_data['brand_numid'] = brand_encoder.fit_transform(labeled_data['brand'])  # Encode labels as numbers - pytorch is happy now s o me too :)
num_classes = len(brand_encoder.classes_)

In [21]:
smallerset = 0.05   #scale down labeled data to a fraction of the original: 
labeled_data = labeled_data.sample(frac=smallerset)

In [22]:
len(labeled_data)

67740

In [23]:
# Custom Dataset for Online Loading
class ImageDataset(Dataset):
    def __init__(self, dataframe, use_bbox, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.usebbox = use_bbox

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['abs_image_path'] 
        label = self.dataframe.iloc[idx]['brand_numid']
        x1 = self.dataframe.iloc[idx]['yolobox_top_left_x']
        y1 = self.dataframe.iloc[idx]['yolobox_top_left_y']
        x2 = self.dataframe.iloc[idx]['yolobox_bottom_right_x']
        y2 = self.dataframe.iloc[idx]['yolobox_bottom_right_y']
        
        # Load image
        image = Image.open(img_path).convert('RGB')
        if self.usebbox and x1 != -1:
            image = image.crop((x1, y1, x2, y2))
        # Apply transformations if any
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [24]:

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing images to 224x224 for pretrained models
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [25]:

# Initialize Dataset and DataLoader
batch_size = 64
dataset = ImageDataset(labeled_data, use_bbox, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [26]:

# Define Model (Using a Pretrained Model)
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(labeled_data['brand'].unique()))  # Adjust output layer for classes
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')


/home/frederic/Documents/automotive_project/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/frederic/Documents/automotive_project/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [28]:

# Online Training Loop
epochs = 5
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [29]:

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    
    for batch_idx, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        print(f'Epoch {epoch+1}/{epochs}, Batch {batch_idx+1}/{len(dataloader)}, Loss: {loss.item():.4f}')

    print(f'Epoch {epoch+1} completed with average loss: {epoch_loss/len(dataloader):.4f}')


Epoch 1/5, Batch 1/1059, Loss: 3.6227
Epoch 1/5, Batch 2/1059, Loss: 3.3292
Epoch 1/5, Batch 3/1059, Loss: 3.3696
Epoch 1/5, Batch 4/1059, Loss: 3.2017
Epoch 1/5, Batch 5/1059, Loss: 2.6576
Epoch 1/5, Batch 6/1059, Loss: 3.0759
Epoch 1/5, Batch 7/1059, Loss: 3.3669
Epoch 1/5, Batch 8/1059, Loss: 3.2764
Epoch 1/5, Batch 9/1059, Loss: 3.2291
Epoch 1/5, Batch 10/1059, Loss: 3.1491
Epoch 1/5, Batch 11/1059, Loss: 3.1991
Epoch 1/5, Batch 12/1059, Loss: 3.0999
Epoch 1/5, Batch 13/1059, Loss: 3.0065
Epoch 1/5, Batch 14/1059, Loss: 3.2927
Epoch 1/5, Batch 15/1059, Loss: 3.0401
Epoch 1/5, Batch 16/1059, Loss: 3.0687
Epoch 1/5, Batch 17/1059, Loss: 3.0317
Epoch 1/5, Batch 18/1059, Loss: 3.1183
Epoch 1/5, Batch 19/1059, Loss: 2.9224
Epoch 1/5, Batch 20/1059, Loss: 2.7711
Epoch 1/5, Batch 21/1059, Loss: 3.0376
Epoch 1/5, Batch 22/1059, Loss: 3.0071
Epoch 1/5, Batch 23/1059, Loss: 3.1286
Epoch 1/5, Batch 24/1059, Loss: 2.9381
Epoch 1/5, Batch 25/1059, Loss: 3.0219
Epoch 1/5, Batch 26/1059, Loss: 3.

In [33]:

# Save Model
torch.save(model.state_dict(), '../../models/pytorch/brand_predictor.pth')
print("Training complete and model saved.")

Training complete and model saved.
